# 🌊 CandorFlow Interactive Demo

## Early Warning System for Training Instabilities

---

### ⚠️ **Important Notice**

**This notebook contains a SIMPLIFIED, EDUCATIONAL demonstration of CandorFlow concepts.**

This is **NOT the full proprietary system**. Many advanced algorithms and features are intentionally excluded, including:
- Universal scaling laws
- Reflexive ridge computations
- Cross-domain invariants
- Advanced control algorithms
- Production optimizations

For information about the complete system, please contact the authors.

---

## 📖 What You'll Learn

In this notebook, you will:

1. ✅ Understand the **λ(t) stability metric** (simplified version)
2. ✅ See how to **monitor training instabilities** in real-time
3. ✅ Learn about **automatic intervention** (rollback + LR reduction)
4. ✅ Visualize **stability patterns** during training
5. ✅ Run a **complete training demo** with intentional instability

Let's get started! 🚀


In [ ]:
# Install dependencies (uncomment if needed)
# !pip install torch transformers numpy matplotlib

# Imports
import sys
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

# Import CandorFlow
from candorflow import compute_lambda, StabilityController
from candorflow.utils import set_seed, get_logger
from candorflow.lambda_metric import compute_lambda_metric_simple

print("✓ All imports successful!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


## Part 1: Understanding λ(t)

The **lambda metric λ(t)** is a stability indicator that tracks training health over time.

### Simplified Version (This Demo)

In this demo, λ(t) is computed as:

```
λ(t) = Var(∇) / Mean(∇)²
```

Where `∇` represents recent gradient norms.

**Interpretation:**
- **Low λ(t)**: Gradients are stable and consistent → Training is healthy
- **High λ(t)**: Gradients are erratic and varying wildly → Instability approaching

### What's Missing (Proprietary)

The full CandorFlow λ(t) includes:
- Jacobian spectral analysis
- Universal scaling laws
- Reflexive ridge computations
- Multi-signal fusion
- Cross-domain invariants

Let's see the simplified metric in action!


In [ ]:
# Create a simple toy model
class TinyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Initialize
set_seed(42)
model = TinyModel()
criterion = nn.CrossEntropyLoss()

print("✓ Created tiny model with 3 layers")
print(f"✓ Total parameters: {sum(p.numel() for p in model.parameters())}")


In [ ]:
# Demonstrate λ(t) computation on stable vs unstable gradients

gradient_history_stable = []
gradient_history_unstable = []

# Simulate stable training
print("Simulating STABLE gradients:")
for i in range(10):
    # Stable: gradients around 1.0 with small noise
    grad_norm = 1.0 + np.random.normal(0, 0.1)
    gradient_history_stable.append(grad_norm)
    print(f"  Step {i}: grad_norm = {grad_norm:.4f}")

lambda_stable = compute_lambda_metric_simple(gradient_history_stable)
print(f"\n✓ Stable λ(t) = {lambda_stable:.4f}\n")

# Simulate unstable training
print("Simulating UNSTABLE gradients:")
for i in range(10):
    # Unstable: wild gradient swings
    grad_norm = 1.0 + np.random.normal(0, 2.0)  # Much larger variance
    gradient_history_unstable.append(grad_norm)
    print(f"  Step {i}: grad_norm = {grad_norm:.4f}")

lambda_unstable = compute_lambda_metric_simple(gradient_history_unstable)
print(f"\n✓ Unstable λ(t) = {lambda_unstable:.4f}")

print(f"\n📊 Comparison: Unstable λ(t) is {lambda_unstable/lambda_stable:.1f}x higher!")


## Part 2: Training with Stability Monitoring

Now let's train a model with CandorFlow monitoring:

1. **Normal training** → λ(t) stays low, no intervention
2. **Inject instability** → λ(t) spikes
3. **Automatic rollback** → Controller restores stability

This demonstrates the basic autopilot functionality.


In [ ]:
# Setup training
set_seed(42)
model = TinyModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Create stability controller
controller = StabilityController(
    threshold=1.5,  # Intervene when λ(t) > 1.5
    checkpoint_dir="../checkpoints_demo",
    lr_reduction_factor=0.5
)

# Training loop
num_steps = 40
gradient_history = []
lambda_values = []
losses = []
actions_taken = []

print("🚀 Starting training with stability monitoring...\n")

for step in range(num_steps):
    # Generate dummy batch
    X = torch.randn(16, 10)
    y = torch.randint(0, 2, (16,))
    
    # Forward pass
    model.train()
    outputs = model(X)
    loss = criterion(outputs, y)
    losses.append(loss.item())
    
    # Compute λ(t)
    lambda_value = compute_lambda(
        model=model,
        loss=loss,
        history_window=10,
        gradient_history=gradient_history
    )
    lambda_values.append(lambda_value)
    
    # Update controller
    action = controller.update(
        lambda_value=lambda_value,
        model=model,
        optimizer=optimizer,
        step=step
    )
    actions_taken.append(action)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    
    # INJECT INSTABILITY after step 25
    if step >= 25:
        # Add large noise to gradients
        for param in model.parameters():
            if param.grad is not None:
                noise = torch.randn_like(param.grad) * 3.0 * (step - 24)
                param.grad.data += noise
    
    optimizer.step()
    
    # Logging
    if step % 5 == 0 or action["action"] != "none":
        status = "🟢" if lambda_value < 1.0 else "🟡" if lambda_value < 1.5 else "🔴"
        print(f"{status} Step {step:2d} | Loss: {loss.item():.4f} | λ(t): {lambda_value:.4f} | {action['action']}")

print("\n✓ Training complete!")

# Summary
summary = controller.get_summary()
print("\n" + "="*50)
print("Summary:")
print("="*50)
print(f"Total interventions: {summary['total_interventions']}")
print(f"Max λ(t): {summary['max_lambda']:.4f}")
print(f"Mean λ(t): {summary['mean_lambda']:.4f}")


## Part 3: Visualize the Results

Let's plot the λ(t) curve to see how the controller responded to instability.


# Plot λ(t) curve with interventions
plt.figure(figsize=(14, 6))

# Plot 1: Lambda curve
plt.subplot(1, 2, 1)
steps = range(len(lambda_values))
plt.plot(steps, lambda_values, linewidth=2, label='λ(t)', color='#2E86AB')
plt.axhline(y=controller.threshold, color='#A23B72', linestyle='--', linewidth=2, label='Threshold')

# Mark interventions
rollback_steps = [i for i, a in enumerate(actions_taken) if a["action"] == "rollback"]
if rollback_steps:
    plt.scatter(
        rollback_steps,
        [lambda_values[i] for i in rollback_steps],
        color='#F18F01',
        s=150,
        marker='o',
        label='Rollback',
        zorder=5,
        edgecolors='black',
        linewidths=2
    )

plt.xlabel('Training Step', fontsize=12)
plt.ylabel('λ(t) - Stability Metric', fontsize=12)
plt.title('CandorFlow Stability Monitoring (Simplified)', fontsize=13, fontweight='bold')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

# Plot 2: Loss curve
plt.subplot(1, 2, 2)
plt.plot(steps, losses, linewidth=2, color='#C73E1D', label='Loss')

if rollback_steps:
    plt.scatter(
        rollback_steps,
        [losses[i] for i in rollback_steps],
        color='#F18F01',
        s=150,
        marker='o',
        label='Rollback',
        zorder=5,
        edgecolors='black',
        linewidths=2
    )

plt.xlabel('Training Step', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss', fontsize=13, fontweight='bold')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/notebook_demo.png', dpi=150, bbox_inches='tight')
print("✓ Saved plot to ../plots/notebook_demo.png")
plt.show()

print("\n📊 Observation:")
print("  • λ(t) was low and stable during normal training")
print("  • Around step 25, we injected gradient noise")
print("  • λ(t) spiked above threshold → Controller intervened")
print("  • Training recovered after rollback + LR reduction")


## Part 4: Key Takeaways

### What We Demonstrated ✅

1. **Early Warning**: λ(t) detected instability *before* complete divergence
2. **Automatic Intervention**: Controller rolled back without manual intervention
3. **Recovery**: Training continued successfully after LR reduction
4. **Simplicity**: Clean API with minimal code changes

### What's Missing from Full System ❌

This demo showed a **basic proof-of-concept**. The full proprietary CandorFlow system includes:

- **Universal Scaling Laws**: λ(t) that works across all model architectures and domains
- **Reflexive Ridge Algorithm**: Advanced mathematical framework for stability
- **Predictive Modeling**: Forecast instabilities before they occur
- **Multi-Signal Fusion**: Combine gradients, loss, activations, and more
- **Cross-Domain Applications**: ECG monitoring, earthquake detection, market analysis
- **Production Optimizations**: Minimal overhead, HPC integration, distributed training
- **Advanced Control**: Adaptive thresholds, temporal smoothing, active inference

### Next Steps 🚀

To run more extensive demos:

```bash
# Full training demo with larger model
python examples/demo_training_loop.py

# Generate publication-quality plots
python examples/demo_plots.py
```

### Learn More 📚

- Read the full README: `../README.md`
- Check the source code: `../candorflow/`
- Contact us for information about the full proprietary system

---

**Thank you for trying CandorFlow!** 🌊


In [ ]:
# Plot λ(t) curve with interventions
plt.figure(figsize=(14, 6))

# Plot 1: Lambda curve
plt.subplot(1, 2, 1)
steps = range(len(lambda_values))
plt.plot(steps, lambda_values, linewidth=2, label='λ(t)', color='#2E86AB')
plt.axhline(y=controller.threshold, color='#A23B72', linestyle='--', linewidth=2, label='Threshold')

# Mark interventions
rollback_steps = [i for i, a in enumerate(actions_taken) if a["action"] == "rollback"]
if rollback_steps:
    plt.scatter(
        rollback_steps,
        [lambda_values[i] for i in rollback_steps],
        color='#F18F01',
        s=150,
        marker='o',
        label='Rollback',
        zorder=5,
        edgecolors='black',
        linewidths=2
    )

plt.xlabel('Training Step', fontsize=12)
plt.ylabel('λ(t) - Stability Metric', fontsize=12)
plt.title('CandorFlow Stability Monitoring (Simplified)', fontsize=13, fontweight='bold')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

# Plot 2: Loss curve
plt.subplot(1, 2, 2)
plt.plot(steps, losses, linewidth=2, color='#C73E1D', label='Loss')

if rollback_steps:
    plt.scatter(
        rollback_steps,
        [losses[i] for i in rollback_steps],
        color='#F18F01',
        s=150,
        marker='o',
        label='Rollback',
        zorder=5,
        edgecolors='black',
        linewidths=2
    )

plt.xlabel('Training Step', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss', fontsize=13, fontweight='bold')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/notebook_demo.png', dpi=150, bbox_inches='tight')
print("✓ Saved plot to ../plots/notebook_demo.png")
plt.show()

print("\n📊 Observation:")
print("  • λ(t) was low and stable during normal training")
print("  • Around step 25, we injected gradient noise")
print("  • λ(t) spiked above threshold → Controller intervened")
print("  • Training recovered after rollback + LR reduction")


---

## 📝 Disclaimer

This notebook demonstrates **simplified, educational implementations** for illustration purposes only.

**This is NOT the full proprietary CandorFlow system.**

The complete system includes proprietary algorithms, universal scaling laws, reflexive computations, and cross-domain capabilities that are protected by pending patents and NOT included in this public demo.

For inquiries about the full system, commercial licensing, or research collaborations, please contact the authors.

**License**: This demo code is provided under the MIT License. See LICENSE file for details.
